# Results from MILP Realtime Computations

In [25]:
import numpy as np
import pandas as pd

from ctmmodels.const import *

In [2]:
IMAGE_PATH = 'graphs/'
DF_PATH = 'experiments/'

In [3]:
time_range = 30
time_ticks = np.arange(0, time_range+1, 1)

parameters = {
    'r_left': 0.25,
    'r_through': 0.5,
    'r_right': 0.25,
    'sat_flow_rate': 1800,
    'time_range': time_range,
    'time_step': 2,
    'g_min': 6,
    'g_max': 20,
    'flow_rate_reduction': 1
}

# Varying parameters: demand, alpha, beta, gamma

## Saving dataframes

In [4]:
def save_df(df, filename):
    df.to_pickle(DF_PATH + filename + ".pkl")

## Loading CSV files

In [5]:
_demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_weights = [
    (0.8, 0.1, 0.1),
    (0.1, 0.8, 0.1),
    (0.1, 0.1, 0.8),
    (0.45, 0.45, 0.1),
    (0.1, 0.45, 0.45),
    (0.45, 0.1, 0.45),
    (0.33, 0.33, 0.33),
    (0, 0, 0),
]

_col_rename = {
    'Runtime': 'runtime',
    'Delay': 'delay',
    'Throughput': 'throughput',
    'ObjValue': 'objective_value'
}

_model_type = {
    (0.8, 0.1, 0.1): 'Delay priority',
    (0.1, 0.8, 0.1): 'Throughput priority',
    (0.1, 0.1, 0.8): 'Flow priority',
    (0.45, 0.45, 0.1): 'Delay-Throughput priority',
    (0.1, 0.45, 0.45): 'Throughput-Flow priority',
    (0.45, 0.1, 0.45): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [12]:
_results_dflist = []

for demand in _demands:
    for weights in _weights:
        if weights == (0,0,0):
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_old.csv'.format(demand))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_old.csv'.format(demand))
        else:
            df_e1 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch1_a{}_b{}_c{}.csv'.format(demand, *weights))
            df_e2 = pd.read_csv(DF_PATH + 'milp-results/results_d{}_epoch2_a{}_b{}_c{}.csv'.format(demand, *weights))

        df_e1 = df_e1.rename(columns=_col_rename)
        df_e1['delay'] = df_e1['delay'].apply(lambda x: 2*x) # IMPORTANT; forgot to add this in the CTM
        df_e1['demand'] = [demand]
        df_e1['new_model'] = weights != (0,0,0)
        df_e1['alpha'] = weights[0]
        df_e1['beta'] = weights[1]
        df_e1['gamma'] = weights[2]
        df_e1['model_type'] = _model_type[weights]
        df_e1['epoch'] = 1
        
        df_e2 = df_e2.rename(columns=_col_rename)
        df_e2['delay'] = df_e2['delay'].apply(lambda x: 2*x)
        df_e2['demand'] = [demand]
        df_e2['new_model'] = weights != (0,0,0)
        df_e2['alpha'] = weights[0]
        df_e2['beta'] = weights[1]
        df_e2['gamma'] = weights[2]
        df_e2['model_type'] = _model_type[weights]
        df_e2['epoch'] = 2
        
        _results_dflist.append(pd.concat([df_e1, df_e2]))

In [13]:
df = pd.concat(_results_dflist)

## Adding initial MILP results

In [15]:
df_initial_results = pd.concat([
    pd.read_pickle(DF_PATH + 'results1.pkl'),
    pd.read_pickle(DF_PATH + 'results2.pkl')
])

In [16]:
df_initial_results.head()

,demand_ns,demand_ew,runtime,delay,throughput,objective_value,alpha,beta,gamma
0,450,450,174.118326,767.272152,51.278481,382.056221,0.80,0.10,0.10
1,450,450,415.136684,791.607595,55.000000,-291.456345,0.10,0.80,0.10
2,450,450,891.892373,801.107595,54.000000,-1585.763682,0.10,0.10,0.80
3,450,450,343.920306,767.272152,51.278481,49.987495,0.45,0.45,0.10
4,450,450,596.610098,791.607595,55.000000,-937.523394,0.10,0.45,0.45


In [17]:
df.head()

,runtime,delay,throughput,objective_value,demand,new_model,alpha,beta,gamma,model_type,epoch
0,3.216363,8911.316456,66.835443,3167.632535,450,True,0.8,0.1,0.1,Delay priority,1
0,2.705916,14393.012658,66.835443,5233.862720,450,True,0.8,0.1,0.1,Delay priority,2
0,2.739941,8653.316456,67.835443,46.513092,450,True,0.1,0.8,0.1,Throughput priority,1
0,3.006180,13110.303797,68.835443,278.233317,450,True,0.1,0.8,0.1,Throughput priority,2
0,3.362435,9995.316456,67.835443,-775.294246,450,True,0.1,0.1,0.8,Flow priority,1


In [19]:
df_initial_results['demand'] = df_initial_results.apply(lambda row: 
                                                        row['demand_ns'] if row['demand_ns'] == row['demand_ew']
                                                        else (int(row['demand_ns']), int(row['demand_ew']))
                                                        , axis=1)

df_initial_results['new_model'] = df_initial_results.apply(lambda row: row['alpha'] != 0, axis=1)

df_initial_results['model_type'] = df_initial_results.apply(lambda row: _model_type[
    (row['alpha'], row['beta'], row['gamma'])
], axis=1)

df_initial_results['epoch'] = 0

df_initial_results['delay'] = df_initial_results['delay'].apply(lambda x: 2*x)

In [20]:
df_initial_results = df_initial_results[[
    'runtime',
    'delay',
    'throughput',
    'objective_value',
    'demand',
    'new_model',
    'alpha',
    'beta',
    'gamma',
    'model_type',
    'epoch'
]]

In [21]:
df = df.rename(columns={'obj-value': 'objective_value'})

In [22]:
df_final = pd.concat([df_initial_results, df]).sort_values(by=['epoch', 'demand', 'model_type'])

In [24]:
save_df(df_final, 'milp-realtime-results')